## basic things

In [1]:
import numpy as np
import math
import cmath
import random
import scipy.linalg

In [2]:
def matr_Ry(angle):
    return np.array([[math.cos(angle/2), -math.sin(angle/2)],
              [+math.sin(angle/2), math.cos(angle/2)]])

In [3]:
def matr_Rx(angle):
    return np.array([[math.cos(angle/2), -1j * math.sin(angle/2)],
              [-1j * math.sin(angle/2), math.cos(angle/2)]])

In [4]:
def matr_Rz(angle):
    return np.array([[math.cos(angle/2) - 1j * math.sin(angle/2), 0],
              [0, math.cos(angle/2) + 1j * math.sin(angle/2)]])

In [5]:
RxRy = matr_Rx(math.pi/2) @ matr_Ry(math.pi/2)
big_U_br1 = 2 * np.kron(RxRy, np.conj(RxRy))
big_U_br2 = 2 * np.kron(np.conj(RxRy), RxRy)

## noising procedures ##

In [6]:
def random_rotation(M):
    phase1 = random.uniform(-math.pi/10, math.pi/10)
    phase2 = random.uniform(-math.pi/10, math.pi/10)
    print ("Noising is made by Rx(", round((phase1/math.pi), 5), "pi) @ U @ Ry(", round((phase2/math.pi), 5), "pi)")
    return matr_Rx(phase1) @ M @ matr_Rx(phase2)

In [7]:
def depolarizing(big_M):
    p = random.uniform(0.25, 0.5)
    #p = 1.0
    PauliY = np.array([[0, -1j],
              [1j, 0]])
    PauliX = np.array([[0, 1],
              [1, 0]])
    PauliZ = np.array([[1, 0],
              [0, -1]])
    print ("Noising is made by depolarising channel with p =", round(p, 5))
    channel = np.diag([1, 1, 1, 1]) * (1 - 3*p/4) + p/4 * (np.kron(PauliY, PauliY) + np.kron(PauliX, PauliX) + np.kron(PauliZ, PauliZ))
    #print("Just in case: depolar looks like\n", channel)
    #print("Just in case: new UxU* is", channel @ big_M)
    return channel @ big_M

In [8]:
def wrong_unitary(M):
    PauliY = np.array([[0, -1j],
              [1j, 0]])
    PauliX = np.array([[0, 1],
              [1, 0]])
    PauliZ = np.array([[1, 0],
              [0, -1]])
    E = np.diag([1, 1])
    pE = random.uniform(-0.2, 0.2)
    pX = random.uniform(-0.1, 0.1)
    pY = random.uniform(-0.1, 0.1)
    pZ = random.uniform(-0.1, 0.1)
    print ("Noising is made by exp(iH) @ U; H =",round(pE, 5) ,"E +",round(pX, 5), "X +",round(pY, 5), "Y +",round(pZ, 5) ,"Z")
    H = pE * E + pX * PauliX + pY * PauliY + pZ * PauliZ
    return scipy.linalg.expm(H * 1j) @ M 
    

## functions for calibration 

In [9]:
def reveal_cal_parameters(M):
    angle = cmath.phase(M[1, 0]) 
    flipped = True if (np.imag(M[1, 1]) > 0) else False
    return angle, flipped

In [10]:
def get_more_precise_angle(M):
    angles = []
    angles.append(cmath.phase(M[1, 0]))
    angles.append(-cmath.phase(M[2, 0]))
    angles.append(cmath.phase(-M[1, 3]))
    angles.append(-cmath.phase(-M[2, 3]))
    angles.append(-cmath.phase(-M[0, 1] * 1j))
    angles.append(cmath.phase(M[0, 2] * 1j))
    angles.append(-cmath.phase(M[3, 1] * 1j))
    angles.append(cmath.phase(-M[3, 2] * 1j))
    angles.append(cmath.phase(M[1, 2] * 1j)/2)
    angles.append(-cmath.phase(-M[2, 1] * 1j)/2)
    #print(angles)
    return np.mean(angles)

In [11]:
def calibrate_set(M_set, angle, flip):
    if (flip == True): #Remember, in flipped scenario angle comes with '-' sign
        M_set = np.conj(M_set) #but if U = [Rz1 @ U0* @ Rz2] that means U* = [Rz1* @ U0 @ Rz2*] and angles here are reversed too
    for i in np.arange(0, len(M_set)): #for implemention makes me cringe
        M_set[i] = matr_Rz(angle) @ M_set[i] @ matr_Rz(-angle)
    return M_set    

## Calibration testing procedures

### Noise 1: random rotations

In [12]:
U = random_rotation(RxRy)
UxU = 2 * np.kron(U, np.conj(U))

Noising is made by Rx( 0.06309 pi) @ U @ Ry( -0.09812 pi)


### Noise 2: depolarizing channel

In [13]:
U = RxRy
UxU = 2 * np.kron(U, np.conj(U))
UxU = depolarizing(UxU)

Noising is made by depolarising channel with p = 0.38978


### Noise 3: random hamiltonian

In [14]:
U = RxRy
U = wrong_unitary(U)
UxU = 2 * np.kron(U, np.conj(U))

Noising is made by exp(iH) @ U; H = 0.19894 E + 0.08904 X + -0.03632 Y + 0.04518 Z


### Testing

In [15]:
angle0, flip = reveal_cal_parameters(UxU)
if (flip): 
    UxU = np.conj(UxU)
angle1 = get_more_precise_angle(UxU)

print ("1th step: we think the angle is", angle0, "and param CONJUGATED is", flip)
print ("2nd step: probably more precise angle is", angle1)

1th step: we think the angle is 0.09647139327279271 and param CONJUGATED is False
2nd step: probably more precise angle is -0.04245606170208531


## Full set calibration

In [16]:
pure_U_set = [matr_Rx(math.pi/2), matr_Ry(math.pi/2)]
rand_rot = random.uniform(-math.pi/2, math.pi/2)
print(rand_rot)

0.3919660308051893


In [17]:
U_set = [matr_Rz(-rand_rot) @ np.conj(matr_Rx(math.pi/2)) @ matr_Rz(rand_rot), matr_Rz(-rand_rot) @ matr_Ry(math.pi/2) @ matr_Rz(+rand_rot)]

In [18]:
U = U_set[0] @ U_set[1]
UxU = 2 * np.kron(U, np.conj(U)) #UxU actually means UxU*
angle0, flip = reveal_cal_parameters(UxU)
if (flip): 
    UxU = np.conj(UxU)
angle1 = get_more_precise_angle(UxU) #angle is reversed if we need to complex conjugate because we made CONJ on UxU 
#CONJ on flipped UxU* is CONJ on ([Rz1 @ U0* @ Rz2] x [Rz1* @ U0 @ Rz2*]]) = [Rz1* @ U0 @ Rz2*] x [Rz1 @ U0* @ Rz2]]
#Because the procedure is trained on U0, this is easier but yields a minus sign
print(angle1, flip) 
#Solution: one more "if" which makes angle1 = -angle1 if (flip). It is for the weak AND does not work properly
new_U_set = calibrate_set(U_set, angle1, flip) #the strong account for reversed angle inside calibration function  

0.39196603080518927 False


In [19]:
print(new_U_set[0].round(8))
print(new_U_set[1].round(8))

[[ 0.70710678+0.j          0.        -0.70710678j]
 [-0.        -0.70710678j  0.70710678-0.j        ]]
[[ 0.70710678+0.j -0.70710678-0.j]
 [ 0.70710678-0.j  0.70710678-0.j]]


In [20]:
pure_U_set

[array([[0.70710678+0.j        , 0.        -0.70710678j],
        [0.        -0.70710678j, 0.70710678+0.j        ]]),
 array([[ 0.70710678, -0.70710678],
        [ 0.70710678,  0.70710678]])]